In [1]:
path_kag = "/content/drive/MyDrive/STD_kaggle.csv"
path_uci = "/content/drive/MyDrive/STDacad_UCI.csv"

In [2]:
import pandas as pd
# df_kag = pd.read_csv(path_kag)
# df_uci = pd.read_csv(path_uci)
# # df_kag.head(5)
# df_uci.head()
import pandas as pd

# path_uci = "/content/drive/MyDrive/STDacad_UCI.csv"
df_uci = pd.read_csv(path_uci, sep=";")   # <-- specify delimiter

print(df_uci.shape)
print(df_uci.head())




(4424, 37)
   Marital status  Application mode  Application order  Course  \
0               1                17                  5     171   
1               1                15                  1    9254   
2               1                 1                  5    9070   
3               1                17                  2    9773   
4               2                39                  1    8014   

   Daytime/evening attendance\t  Previous qualification  \
0                             1                       1   
1                             1                       1   
2                             1                       1   
3                             1                       1   
4                             0                       1   

   Previous qualification (grade)  Nacionality  Mother's qualification  \
0                           122.0            1                      19   
1                           160.0            1                       1   
2              

In [12]:
df_kag = pd.read_csv(path_kag)
print(df_kag.head(5))
print(df_kag.shape)


  School Gender  Age Address Family_Size Parental_Status  Mother_Education  \
0     GP      F   18       U         GT3               A                 4   
1     GP      F   17       U         GT3               T                 1   
2     GP      F   15       U         LE3               T                 1   
3     GP      F   15       U         GT3               T                 4   
4     GP      F   16       U         GT3               T                 3   

   Father_Education Mother_Job Father_Job  ... Free_Time Going_Out  \
0                 4    at_home    teacher  ...         3         4   
1                 1    at_home      other  ...         3         3   
2                 1    at_home      other  ...         3         2   
3                 2     health   services  ...         2         2   
4                 3      other      other  ...         3         2   

   Weekend_Alcohol_Consumption  Weekday_Alcohol_Consumption  Health_Status  \
0                            1  

In [4]:
import pandas as pd
import numpy as np

# ===========================
# Paths
# ===========================
path_uci = "/content/drive/MyDrive/STDacad_UCI.csv"
path_kag = "/content/drive/MyDrive/STD_kaggle.csv"

# ===========================
# Load datasets
# ===========================
df_uci = pd.read_csv(path_uci, sep=";")
df_kag = pd.read_csv(path_kag)

print("UCI shape:", df_uci.shape)
print("Kaggle shape:", df_kag.shape)

# ===========================
# Mapping helper functions
# ===========================
def normalize_to_100(x, max_val):
    return (x / max_val) * 100 if pd.notna(x) else np.nan

def normalize_cgpa(x, max_val):
    return (x / max_val) * 10 if pd.notna(x) else np.nan

def map_gender(val):
    if str(val).strip().lower() in ["m", "male", "1"]:
        return "M"
    elif str(val).strip().lower() in ["f", "female", "0"]:
        return "F"
    else:
        return "O"

def map_fee(val):
    return "on_time" if str(val).strip().lower() in ["1", "yes", "on_time", "paid"] else "delayed"

def map_target_uci(val):
    return "dropout" if str(val).lower() == "dropout" else "not_dropout"

def map_target_kag(val):
    return "dropout" if str(val).lower() in ["true", "1", "yes", "dropout"] else "not_dropout"

# ===========================
# Process UCI dataset
# ===========================
uci = pd.DataFrame({
    "age": df_uci["Age at enrollment"],
    "gender": df_uci["Gender"].map(map_gender),
    "nationality": df_uci["Nacionality"],
    "highschool_score": df_uci["Previous qualification (grade)"].apply(lambda x: normalize_to_100(x, 200)),
    "entrance_exam_score_normalized": df_uci["Admission grade"].apply(lambda x: normalize_to_100(x, 200)),
    "department": df_uci["Course"],
    "admission_type": df_uci["Application mode"],
    "attendance_pct": (df_uci["Curricular units 1st sem (approved)"] / df_uci["Curricular units 1st sem (enrolled)"]) * 100,
    "current_sem_cgpa": df_uci["Curricular units 1st sem (grade)"].apply(lambda x: normalize_cgpa(x, 20)),
    "aggregate_cgpa": ((df_uci["Curricular units 1st sem (grade)"] + df_uci["Curricular units 2nd sem (grade)"]) / 2).apply(lambda x: normalize_cgpa(x, 20)),
    "backlogs_count": df_uci["Curricular units 1st sem (enrolled)"] - df_uci["Curricular units 1st sem (approved)"],
    "family_income_bracket": np.nan,  # not present
    "parent_education": df_uci[["Mother's qualification", "Father's qualification"]].max(axis=1),
    "scholarship_status": df_uci["Scholarship holder"].map(lambda x: "none" if x == 0 else "scholarship"),
    "fee_payment_status": df_uci["Tuition fees up to date"].map(map_fee),
    "residence_type": df_uci["Daytime/evening attendance\t"].map(lambda x: "day_scholar" if x == 1 else "hostel"),
    "commute_distance_km": np.nan,  # not present
    "target_status": df_uci["Target"].map(map_target_uci)
})

# ===========================
# Process Kaggle dataset
# ===========================
kag = pd.DataFrame({
    "age": df_kag["Age"],
    "gender": df_kag["Gender"].map(map_gender),
    "nationality": df_kag["School"],  # proxy for now
    "highschool_score": df_kag["Grade_1"].apply(lambda x: normalize_to_100(x, 20)),
    "entrance_exam_score_normalized": df_kag["Final_Grade"].apply(lambda x: normalize_to_100(x, 20)),
    "department": df_kag.get("Department", np.nan),
    "admission_type": df_kag.get("AdmissionType", np.nan),
    "attendance_pct": df_kag.get("AttendancePercentage", 100 - df_kag["Number_of_Absences"]*2).clip(0, 100),
    "current_sem_cgpa": df_kag["Final_Grade"].apply(lambda x: normalize_cgpa(x, 20)),
    "aggregate_cgpa": df_kag[["Grade_1", "Grade_2", "Final_Grade"]].mean(axis=1).apply(lambda x: normalize_cgpa(x, 20)),
    "backlogs_count": df_kag.get("NumberOfBacklogs", np.nan),
    "family_income_bracket": df_kag.get("FamilyIncome", np.nan),
    "parent_education": df_kag[["Mother_Education", "Father_Education"]].max(axis=1),
    "scholarship_status": df_kag.get("Scholarship", np.nan),
    "fee_payment_status": df_kag.get("FeePayment", np.nan).map(map_fee) if "FeePayment" in df_kag else np.nan,
    "residence_type": df_kag.get("ResidenceType", np.nan),
    "commute_distance_km": df_kag.get("CommuteDistance", np.nan),
    "target_status": df_kag["Dropped_Out"].map(map_target_kag)
})

# ===========================
# Merge into one major dataset
# ===========================
df_major = pd.concat([uci, kag], ignore_index=True)

print("Final unified dataset:", df_major.shape)
print(df_major.head())
print(df_major.isna().sum())

# Columns that are dataset-specific
uci_only = ["department", "admission_type", "backlogs_count",
            "scholarship_status", "fee_payment_status", "residence_type"]

kag_only = ["family_income_bracket", "commute_distance_km"]

# Add missing flags for each
for col in uci_only + kag_only:
    flag_col = col + "_missing"
    df_major[flag_col] = df_major[col].isna().astype(int)

print(df_major.filter(like="_missing").head())


UCI shape: (4424, 37)
Kaggle shape: (649, 34)
Final unified dataset: (5073, 18)
   age gender nationality  highschool_score  entrance_exam_score_normalized  \
0   20      M           1              61.0                           63.65   
1   19      M           1              80.0                           71.25   
2   19      M           1              61.0                           62.40   
3   20      F           1              61.0                           59.80   
4   45      F           1              50.0                           70.75   

   department  admission_type  attendance_pct  current_sem_cgpa  \
0       171.0            17.0             NaN          0.000000   
1      9254.0            15.0      100.000000          7.000000   
2      9070.0             1.0        0.000000          0.000000   
3      9773.0            17.0      100.000000          6.714286   
4      8014.0            39.0       83.333333          6.166667   

   aggregate_cgpa  backlogs_count  family_

In [5]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.3 MB/s eta 0:00:00


In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from catboost import CatBoostClassifier, Pool

# ============================
# 1. Identify leakage columns
# ============================
leakage_cols = [
    "aggregate_cgpa",
    "current_sem_cgpa",
    "backlogs_count",
    "attendance_pct",     # if final, not ongoing
    "fee_payment_status"  # if final, not ongoing
]

# ============================
# 2. Target
# ============================
TARGET = "target_status"

# ============================
# 3. Full dataset (upper bound)
# ============================
X_full = df_major.drop(columns=[TARGET])
y = df_major[TARGET].astype(str)

# Categorical cols (from earlier discussion, minus department)
categorical_cols = [
    "gender",
    "nationality",
    "admission_type",
    "family_income_bracket",
    "parent_education",
    "scholarship_status",
    "fee_payment_status",
    "residence_type"
]
categorical_cols = [c for c in categorical_cols if c in X_full.columns]

for c in categorical_cols:
    X_full[c] = X_full[c].astype(str)

X_train_full, X_test_full, y_train, y_test = train_test_split(
    X_full, y, test_size=0.2, stratify=y, random_state=42
)

cat_idx_full = X_train_full.columns.get_indexer(categorical_cols).tolist()
train_pool_full = Pool(X_train_full, y_train, cat_features=cat_idx_full)
test_pool_full  = Pool(X_test_full,  y_test,  cat_features=cat_idx_full)

# ============================
# 4. Early-warning dataset
# ============================
X_safe = df_major.drop(columns=[TARGET] + leakage_cols, errors="ignore")

categorical_safe = [c for c in categorical_cols if c in X_safe.columns]
for c in categorical_safe:
    X_safe[c] = X_safe[c].astype(str)

X_train_safe, X_test_safe, _, _ = train_test_split(
    X_safe, y, test_size=0.2, stratify=y, random_state=42
)

cat_idx_safe = X_train_safe.columns.get_indexer(categorical_safe).tolist()
train_pool_safe = Pool(X_train_safe, y_train, cat_features=cat_idx_safe)
test_pool_safe  = Pool(X_test_safe,  y_test,  cat_features=cat_idx_safe)

# ============================
# 5. Train CatBoost
# ============================
def train_and_eval(train_pool, test_pool, X_test, label="Model"):
    model = CatBoostClassifier(
        iterations=1000,
        depth=8,
        learning_rate=0.05,
        loss_function="Logloss",
        eval_metric="F1",
        random_seed=42,
        l2_leaf_reg=5.0,
        min_data_in_leaf=30,
        # subsample=0.8,
        bootstrap_type="Bayesian",
        # leaf_estimation_iterations=10,
        # leaf_estimation_method="Newton",
        task_type="GPU",
        # thread_count=-1,
        verbose=100
    )
    model.fit(train_pool, eval_set=test_pool, use_best_model=True)

    y_pred = model.predict(X_test)
    y_pred_class = [p[0] if isinstance(p, list) else p for p in y_pred]

    print(f"\n=== {label} ===")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred_class))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_class))

    return model

# Train both
model_full = train_and_eval(train_pool_full, test_pool_full, X_test_full, label="Full Dataset (with leakage)")
model_safe = train_and_eval(train_pool_safe, test_pool_safe, X_test_safe, label="Early-Warning (no leakage)")


0:	learn: 0.9148936	test: 0.9093298	best: 0.9093298 (0)	total: 180ms	remaining: 2m 59s
100:	learn: 0.9305556	test: 0.9188464	best: 0.9188464 (89)	total: 10.1s	remaining: 1m 29s
200:	learn: 0.9345127	test: 0.9192463	best: 0.9226631 (190)	total: 20s	remaining: 1m 19s
300:	learn: 0.9404132	test: 0.9202703	best: 0.9237002 (248)	total: 28.7s	remaining: 1m 6s
400:	learn: 0.9444159	test: 0.9194313	best: 0.9237002 (248)	total: 39s	remaining: 58.2s
500:	learn: 0.9493324	test: 0.9214092	best: 0.9237002 (248)	total: 49.2s	remaining: 49s
600:	learn: 0.9524953	test: 0.9185889	best: 0.9237002 (248)	total: 1m	remaining: 40.3s
700:	learn: 0.9558950	test: 0.9189925	best: 0.9237002 (248)	total: 1m 10s	remaining: 30.1s
800:	learn: 0.9582689	test: 0.9203540	best: 0.9237002 (248)	total: 1m 19s	remaining: 19.8s
900:	learn: 0.9594502	test: 0.9189925	best: 0.9237002 (248)	total: 1m 29s	remaining: 9.87s
999:	learn: 0.9611550	test: 0.9203540	best: 0.9237002 (248)	total: 1m 40s	remaining: 0us
bestTest = 0.923700